## Create database

In [11]:
import sqlite3
from sqlite3 import Error

#https://www.sqlitetutorial.net/sqlite-python/

def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()

if __name__ == '__main__':
    #so cria o arquivo se a pasta sqlite e db existirem
    #se ao inves de passar o endereco do arquivo, eu passar :memory: o python cria um arquivo na memoria ram
    conn = create_connection(r"C:\sqlite\db\pythonsqlite.db")
    print(conn)

2.6.0
None


## Cria tabela estrela

In [13]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

In [ ]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [ ]:
def main():
    database = "C:\sqlite\db\pythonsqlite.db"

    sql_create_vacinacao_covid_brasil = """ CREATE TABLE IF NOT EXISTS vacinacao_covid_brasil (
                                        FOREIGN KEY (paciente_fk_id) REFERENCES paciente(paciente_id),
                                        FOREIGN KEY(vacina_codigo_fk_id) REFERENCES vacina(vacina_codigo),
                                        FOREIGN KEY(estabelecimento_razao_social_fk_id) REFERENCES estabelecimento(estabelecimento_razao_social)
                                    ); """

    sql_create_paciente_table = """ CREATE TABLE IF NOT EXISTS paciente (
                                        paciente_id text PRIMARY KEY,
                                        document_id text,
                                        paciente_idade integer,
                                        paciente_datanascimento text,
                                        paciente_enumsexobiologico char,
                                        paciente_racacor_codigo integer,
                                        paciente_racacor_valor text,
                                        paciente_endereco_coibgemunicipio integer,
                                        paciente_endereco_copais integer,
                                        paciente_endereco_nmmunicipio text,
                                        paciente_endereco_nmpais text,
                                        paciente_endereco_uf text,
                                        paciente_endereco_cep text,
                                        paciente_nacionalidade_enumnacionalidade char
                                    );"""

    sql_create_vacina_table = """ CREATE TABLE IF NOT EXISTS vacina (
                                        vacina_codigo integer PRIMARY KEY,
                                        vacina_grupoatendimento_codigo integer,
                                        vacina_grupoatendimento_nome text,
                                        vacina_categoria_codigo integer,
                                        vacina_categoria_nome text,
                                        vacina_lote text,
                                        vacina_fabricante_nome text,
                                        vacina_fabricante_referencia text,
                                        vacina_dataaplicacao text,
                                        vacina_descricao_dose text,
                                        vacina_codigo integer,
                                        vacina_numdose integer,
                                        vacina_nome text
                                    );"""

    sql_create_estabelecimento_table = """ CREATE TABLE IF NOT EXISTS estabelecimento (
                                        estabelecimento_municipio_codigo integer PRIMARY KEY,
                                        estabelecimento_razaosocial text,
                                        estabelecimento_valor integer,
                                        estabelecimento_nofantasia text,
                                        estabelecimento_municipio_nome text,
                                        estabelecimento_uf text
                                    );"""

    # create a database connection
    conn = create_connection(database)

    # create tables
    if conn is not None:
        # create historico table
        create_table(conn, sql_create_vacinacao_covid_brasil)
    else:
        print("Error! cannot create the database connection.")

In [ ]:
if __name__ == '__main__':
    main()